In [1]:
import os
import logging
from typing import Any, Set


from automata.config.config_base import AgentConfigName
from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding import SymbolCodeEmbedding
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)

    
logger = logging.getLogger(__name__)

In [2]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "automata"
py_module_loader.initialize(f"..", project_name) # ! Can replace with your own project


In [3]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

In [4]:
symbol_graph.is_synchronized = True
len(symbol_graph.get_sorted_supported_symbols()),len(code_embedding_db)

(8327, 971)

In [5]:
symbol_graph.get_sorted_supported_symbols()[-1]

Symbol(scip-python python automata 8d461ca2e41b857ff2e7ee248f989ff6cffc6a15 `tests.utils.factories`/symbol_search_live()., scip-python, Package(python automata 8d461ca2e41b857ff2e7ee248f989ff6cffc6a15), (Descriptor(tests.utils.factories, 1), Descriptor(symbol_search_live, 4)))

In [6]:
code_embedding_db.get_all_ordered_embeddings()[-1].symbol

Symbol(scip-python python automata 8d461ca2e41b857ff2e7ee248f989ff6cffc6a15 `automata.tools.tool_executor`/ToolExecutor#execute()., scip-python, Package(python automata 8d461ca2e41b857ff2e7ee248f989ff6cffc6a15), (Descriptor(automata.tools.tool_executor, 1), Descriptor(ToolExecutor, 2), Descriptor(execute, 4)))

In [7]:
dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)


symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get('symbol_code_embedding_handler')


In [8]:
raw_ranks = symbol_search.symbol_rank.get_ordered_ranks()
print(f"Demonstrating SymbolRank (on {project_name}) \n")
for i,rank in enumerate(raw_ranks[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

100%|██████████| 971/971 [00:22<00:00, 42.51it/s] 


Demonstrating SymbolRank (on automata) 

rank 0 = automata.symbol.symbol_base.Symbol with rank 0.005
rank 1 = automata.agent.openai_agent.OpenAIAutomataAgent with rank 0.004
rank 2 = automata.singletons.github_client.GitHubClient with rank 0.004
rank 3 = automata.core.base.database.vector_database.ChromaVectorDatabase with rank 0.003
rank 4 = automata.singletons.dependency_factory.DependencyFactory with rank 0.003
rank 5 = automata.core.base.database.vector_database.VectorDatabaseProvider with rank 0.003
rank 6 = automata.core.base.database.vector_database.JSONVectorDatabase with rank 0.003
rank 7 = automata.symbol.graph.symbol_graph.SymbolGraph with rank 0.003
rank 8 = automata.symbol_embedding.vector_databases.ChromaSymbolEmbeddingVectorDatabase with rank 0.003
rank 9 = automata.eval.agent.code_writing_eval.CodeWritingAction with rank 0.003


In [9]:
query = "How do I create an agent?"

embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')
code_embeddings = symbol_code_embedding_handler.get_all_ordered_embeddings()

print(f"Demonstrating Similarity Search (on {project_name}) for query = {query} \n")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with distance {rank[1]:.3f}")

Demonstrating Similarity Search (on automata) for query = How do I create an agent? 

rank 0 = automata.agent.agent.Agent with distance 0.773
rank 1 = automata.tasks.task_executor.IAutomataTaskExecution._build_agent with distance 0.769
rank 2 = automata.agent.openai_agent.OpenAIAutomataAgent with distance 0.766
rank 3 = automata.cli.scripts.run_agent.main with distance 0.765
rank 4 = automata.config.config_base.AgentConfigBuilder.build with distance 0.764
rank 5 = automata.core.run_handlers.create_task with distance 0.763
rank 6 = automata.config.openai_config.OpenAIAutomataAgentConfig.get_llm_provider with distance 0.761
rank 7 = automata.agent.agent.Agent._setup with distance 0.760
rank 8 = automata.tools.agent_tool_factory.AgentToolFactory with distance 0.759
rank 9 = automata.core.run_handlers.run_setup with distance 0.756


In [10]:
symbol_rank_search_results = symbol_search.get_symbol_rank_results(query)
print(f"Demonstrating SymbolRank (on {project_name}) for query = {query} \n")
for i,rank in enumerate(symbol_rank_search_results[0:10]):
    print(f"rank {i} = {rank[0].dotpath} with rank {rank[1]:.3f}")

Demonstrating SymbolRank (on automata) for query = How do I create an agent? 

rank 0 = automata.agent.openai_agent.OpenAIAutomataAgent with rank 0.010
rank 1 = automata.agent.agent.Agent with rank 0.009
rank 2 = automata.config.openai_config.OpenAIAutomataAgentConfigBuilder with rank 0.006
rank 3 = automata.cli.scripts.run_agent.main with rank 0.005
rank 4 = automata.tools.agent_tool_factory.AgentToolFactory with rank 0.005
rank 5 = automata.config.config_base.AgentConfigBuilder with rank 0.005
rank 6 = automata.config.config_base.AgentConfig with rank 0.005
rank 7 = automata.config.openai_config.OpenAIAutomataAgentConfig with rank 0.005
rank 8 = automata.tasks.task_executor.IAutomataTaskExecution._build_agent with rank 0.005
rank 9 = automata.singletons.github_client.GitHubClient with rank 0.004
